In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from numpy import logical_and as npand
from numpy import logical_or as npor
import matplotlib

In [ ]:
from sympy import (symbols, pi, I, E, cos, sin, exp, tan, simplify, expand, factor, collect,
                   apart, cancel, expand_trig, diff, Derivative, Function, integrate, limit,
                   series, Eq, solve, dsolve, Matrix, N, preorder_traversal, Float, solve_linear_system,
                   eye, zeros, lambdify, Symbol,hessian)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting

In [ ]:
from matplotlib import animation, rc
import matplotlib.patches as patches
from matplotlib.transforms import Affine2D
from IPython.display import HTML

In [ ]:
from optibot.schemes import interpolated_array,trap_interp, trapz_step, integrate_trapz

In [ ]:
from optibot.numpy import trapz_integrate, num_derivative

In [ ]:
from optibot.schemes import is2d
def F_ej(x,u,params):
    if is2d(x):
        f = np.zeros_like(x)
        f[:,0] = x[:,2]
        f[:,1] = x[:,3]
        f[:,2] = u[:,0]
        f[:,3] = u[:,1]
    else:
        f = np.zeros_like(x)
        f[0] = x[2]
        f[1] = x[3]
        f[2] = u[0]
        f[3] = u[1]
    return f

In [ ]:
_x_0 = np.array([0.,0.,0.,0.])
_ej_u = np.array([[1.,1.],[1.,1.]])
_x_1 = trapz_step(_x_0, _ej_u[0], _ej_u[1], F_ej, 1., [0.,])

In [ ]:
_x_1

In [ ]:
_ej_x = np.array([_x_0, _x_1])

_ej_x

In [ ]:
_X_interp, _U_interp = interpolated_array(_ej_x, _ej_u, F_ej, 1.,
                                  np.array([0,0.5,1]), [1], 'trapz')
_X_interp

In [ ]:
_ej_u_2 = np.sin(np.linspace([0,0], [20,20], 51))
_ej_x_2 = np.array(integrate_trapz(
    _x_0,
    _ej_u_2,
    F_ej,
    0.1,
    [1,]
))[:-1]
_ej_x_2.shape

In [ ]:
_ej_x_interp, _ej_u_interp = interpolated_array(_ej_x_2, _ej_u_2, F_ej, 0.1,
                                  np.linspace(0, 5,101), [1,], 'trapz')

In [ ]:
plt.figure(figsize=[50,20])
aa = 2
ej_t_array = np.linspace(0, 5,51)
new_ej_t_array = np.linspace(0, 5,101)
plt.plot(new_ej_t_array, _ej_x_interp[:,aa], 'b')
plt.plot(ej_t_array, _ej_x_2[:,aa], 'r:')
plt.plot(ej_t_array, _ej_x_2[:,aa], 'g:')
plt.plot(ej_t_array, _ej_x_2[:,aa], 'ko')
plt.vlines(new_ej_t_array, -0.1, 0.6)

In [ ]:
h = 0.1

In [ ]:
accelerations = F_ej(_ej_x_2, _ej_u_2,[1,])
accelerations

In [ ]:
_err_1 = accelerations[:,2] - num_derivative(accelerations[:,0], h)
_err_2 = accelerations[:,0] - _ej_x_2[:,2]
_err_3 = _ej_x_2[:,2] - num_derivative(_ej_x_2[:,0],h)

In [ ]:
plt.figure(figsize=[15,7])
plt.plot(ej_t_array,_err_1, label = f'error 1')
plt.plot(ej_t_array,_err_2, label = f'error 2')
plt.plot(ej_t_array,_err_3, label = f'error 3')
plt.plot([0, ej_t_array[-1]], [0,0], 'k', lw=1, ls = ':')
plt.legend()

In [ ]:
_err_4 = trapz_integrate(accelerations[:,2], ej_t_array) - accelerations[:,0]
_err_5 = trapz_integrate(_ej_x_2[:,2], ej_t_array) - _ej_x_2[:,0]

In [ ]:
plt.figure(figsize=[15,7])
plt.plot(ej_t_array,_err_4, label = f'error 4')
plt.plot(ej_t_array,_err_5, label = f'error 5')
plt.plot([0, ej_t_array[-1]], [0,0], 'k', lw=1, ls = ':')
plt.legend()

In [ ]:
import casadi as cas
import numpy as np

In [ ]:
import optibot.schemes as sch

In [ ]:
import functools
from numpy import array, expand_dims
from optibot.schemes import vec_len, euler_step
from scipy.optimize import root
from copy import copy

def coherent_dimensions(func):
    '''
    Adapts input variables to ensure that they are compatible
    with functions of structure integrate_x(x_0, u, F, dt, params)

    Parameters
    ----------
    func : Function
        Integration function whose structure is F(x_0, u, F, dt, params).
    -------
    Function
        The same function, but with additional comprobations
        that the input variables are coherent.

    '''
    @functools.wraps(func)
    def wrapper_decorator(x_0, u, F, dt, params):
        x_0 = array(x_0, dtype=float)
        u = array(u, dtype=float)
        
        # If u was a number, it will produce errors later
        # while trying to iterate over it. We have to to convert it
        # into a 1D array of lenght 1 
        if u.size == 1 and u.shape == ():
            u = expand_dims(u, axis=0)
            
        # x_0 is the initial state and must be 1D:
        if not (len(x_0.shape) == 1 or (len(x_0.shape) == 2 and x_0.shape[0] == 1)):
            raise ValueError(
                f"x_0 must be a 1D array, but instead its shape is {x_0.shape}"
            )
        # If x_0 is a 2D array of one line, we have to convert it
        # to a normal 1D array so that the integration is coherent.
        if len(x_0.shape) == 2:
            x_0 = x_0[0]
        # If u is 1D but the problem has more than 1 q,
        # it means that it corresponds to only one step
        if len(u.shape) == 1 and x_0.shape[0] != 2:
            u = expand_dims(u, axis=0)
        value = func(x_0, u, F, dt, params)
        return value

    return wrapper_decorator

def trapz_mod_opti_step(x_n, x, u, u_n, F, dt, params):
    dim = vec_len(x) // 2
    f = F(x, u, params)[dim:]
    f_n = F(x_n, u_n, params)[dim:]
    res = copy(x)
    res[dim:] = x[dim:] + dt / 2 * (f + f_n) - x_n[dim:]
    res[:dim] = x[:dim] + dt * x[dim:] + dt ** 2 / 6 * (f_n + 2 * f) - x_n[:dim]
    return res


def trapz_mod_step(x, u, u_n, F, dt, params):
    x_0 = euler_step(x, u, F, dt, params)
    x_n = root(trapz_mod_opti_step, x_0, (x, u, u_n, F, dt, params))
    return x_n.x

@coherent_dimensions
def integrate_trapz_mod(x_0, u, F, dt, params):
    x = [
        x_0,
    ]
    for ii in range(0, vec_len(u) - 1):
        #print('x: ', x[-1],'u: ', u[ii], 'un: ', u[ii + 1])
        x_i = trapz_mod_step(x[-1], u[ii], u[ii + 1], F, dt, params)
        #print('x_i: ', x_i)
        x.append(x_i)
    #print('x: ', x[-1],'u: ', u[-1], 'un: ', u[-1])
    x_i = trapz_mod_step(x[-1], u[-1], u[-1], F, dt, params)
    x.append(x_i)
    return array(x)

In [ ]:
cases = [
    [[0,1], 1],
    [[0,1], [0,1]],
    [[[0,1],], [0,1]],
    [[0,0,1,1], [0,1]],
    [[0,0,1,1], [[0,1],[2,3]]],
]

In [ ]:
F = sch.expand_F(lambda x, u, params: u, mode="numpy")
params = []

In [ ]:
sch.trapz_mod_opti_step(
    np.array([0.5, 1.]),
    np.array([0., 1.]),
    np.array([0.]),
    np.array([1.]),
    F,
    0.5,
    params
)

In [ ]:
for x_test, u_test in cases:
    #print('\n\nCase\nx_0: ',x_test, '\nu: ', u_test, '\n\n')
    res = sch.integrate_hs_mod(x_test, u_test, F, 0.5, params)
    print('[', end='')
    for ii in res:
        print('[', end='')
        for jj in ii:
            print(jj, ',', end='')
        print('],', end='')
    print('],')

In [ ]:
integrate_funcs = [sch.trap_interp,
                       sch.trap_mod_interp, sch.hs_interp, sch.hs_mod_interp]

In [ ]:
import casadi as cas
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import optibot.schemes as sch

In [ ]:
def generate_restrictions_parameters():
    schemes = ['trapz', 'trapz_mod', 'hs', 'hs_mod', 'hs_scipy']
    integrate_funcs = [sch.integrate_trapz, sch.integrate_trapz_mod, sch.integrate_hs, sch.integrate_hs_mod, sch.integrate_hs]
    x_0 =[np.array([0.,1.]), np.array([0.,1.,2.,3.])]
    u = [np.array([3.,4.]), np.array([[4., 5.],[6., 7.]])]
    
    test_cases = []
    for ii in range(len(schemes)):
        fun = schemes[ii]
        int_fun = integrate_funcs[ii]
        for jj in range(len(x_0)):
            x_0_case = x_0[jj]
            u_case = u[jj]
            test_cases.append((fun, int_fun, x_0_case, u_case))
    return test_cases

In [ ]:
test_cases = generate_restrictions_parameters()

In [ ]:
test_cases = generate_restrictions_parameters()
for ii in range(len(test_cases)):
    scheme, func_int, x_0, U = test_cases[ii]
    print(f'F_test {ii}: {scheme}, F_int: {func_int.__name__}, x_0: {x_0}, u: {U}')
    F = sch.expand_F(lambda x, u, params: u)
    h = 0.5
    t_array = np.array([h, h*1.5, (2*h)-1e-8])
    params = []
    X = func_int(x_0, U, F, h, params)
    interp_X, interp_U = sch.interpolated_array(X, U, F, h, t_array, params, scheme)
    print('\nInterpolated:', interp_X[0] - X[1], interp_X[2] - X[2],'\n', interp_X[1],'\n')

In [ ]:
np.concatenate([np.array([0.34895833,2]), np.array([1.8125,3])])

In [ ]:
F((np.array([0.34895833]), np.array([1.8125])),3.5, params)

In [ ]:
sch.hs_midpoint(x_0, x_n, u[0], u[1], tau, F, dt, params)

In [ ]:
residue